In [1]:
import numpy as np
import os
from nd2reader import ND2Reader
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib_scalebar.scalebar import ScaleBar
import scipy.ndimage as ndi
from skimage.filters import rank 
from scipy.signal import butter, lfilter
import scipy.misc as misc
from skimage import exposure
from PIL import Image
import colorcet as cc
from scipy.misc import toimage
import matplotlib.colors as clr
import pandas as pd
import seaborn as sns
from scipy import stats
import cv2
from pyspark import SparkContext, SparkConf
import skimage
from pathlib import Path

def mean2(x):
    y = np.sum(x) / np.size(x);
    return y
def corr2(a,b):
    a = a - mean2(a)
    b = b - mean2(b)

    r = (a*b).sum() / np.sqrt((a*a).sum() * (b*b).sum());
    return r
from copy import deepcopy
from skimage.filters import threshold_otsu,threshold_li
import glob
from IPython.display import clear_output
import scipy.io as spio
import pims
from skimage.measure import label, regionprops
from skimage.color import label2rgb
import scipy

In [2]:
expdir='/Volumes/ukps-all$/user/tdanko/Imaging/1_ECM/Fixed culture/Leica confocal/'
exp="20190515_tnr550blocked_int1fitcblocked_plus0h_"

In [3]:
results = {"filename":[],
          "exp condition":[],
          "ecmchannelno":[],
          "endosomechannelno":[],
           "Manual ECM spots mask":[],
           "Manual endosome spots mask":[],
          }
           
path = os.path.dirname(expdir)  
outputFolder = os.path.dirname(expdir + "/new/")
filelist = []
for root, _, filenames in os.walk(path):
     for filename in filenames:
         filelist.append(os.path.join(root, filename))      
for i in range(0,len(filelist)):
    if "_analyse" in str(filelist[i]):
        if "2019" in str(filelist[i]):
            if ("posthoc" in str(filelist[i]))==False:
                if (exp in str(filelist[i]))==True:
                #if (protein in str(filelist[i]))==True:
                    if filelist[i].endswith(".tif"): 
                        if str(filelist[i][:-12]) not in str(results["filename"]):
                            results["filename"].append(filelist[i][:-12])
                            results["exp condition"].append(filelist[i][:-12].split('/')[-2])
                            channeldir="/".join(filelist[i][:-12].split("/")[:-1])
                            exec(open(channeldir+"/channels.txt").read())
                            results["ecmchannelno"].append(ecmchannelno)
                            results["endosomechannelno"].append(endosomechannelno)
                                        

## Loading a random image

In [1]:
randomno=np.random.randint(1,len(results["filename"]))
ecm_channel_no=results["ecmchannelno"][randomno]
endosome_channel_no=results["endosomechannelno"][randomno]
#synapse_channel_no=results["synapsechannelno"][randomno]

ecm_image_name=results["filename"][randomno]+"0_ch"+str(ecm_channel_no-1)+"_t0.tif"
endosome_image_name=results["filename"][randomno]+"0_ch"+str(endosome_channel_no-1)+"_t0.tif"
#synapse_image_name=results["filename"][randomno]+"0_ch"+str(synapse_channel_no-1)+"_t0.tif"

print("Image name:",results["filename"][randomno].split('/')[-1])
print("Experimental condition:",results["exp condition"][randomno])
ecm_image=pims.open(ecm_image_name)[0]
endosome_image=pims.open(endosome_image_name)[0]
#synapse_image=pims.open(synapse_image_name)[0]
   
figure=plt.figure(figsize=(15,15))
figure.add_subplot(131)
plt.imshow(ecm_image,interpolation='none',cmap='hot')
plt.title('ECM raw')
plt.grid(visible=False)
plt.xticks(visible=False)
plt.yticks(visible=False)
scalebar = ScaleBar(40, 'nm', dimension='si-length',box_alpha=0,location='lower right',length_fraction=0.2,
                       font_properties={'size':16,'weight':'bold'},color='white',border_pad=0.5) # 1 pixel = 0.2 meter
plt.gca().add_artist(scalebar)
#figure.add_subplot(132)
#plt.imshow(ecm_image,interpolation='none',cmap='hot',vmin=3*np.min(ecm_image),vmax=0.4*np.max(ecm_image))
#plt.title('ECM scaled')
#plt.grid(visible=False)
#plt.xticks(visible=False)
#plt.yticks(visible=False)
#scalebar = ScaleBar(40, 'nm', dimension='si-length',box_alpha=0,location='lower right',length_fraction=0.2,
 #                      font_properties={'size':16,'weight':'bold'},color='white',border_pad=0.5) # 1 pixel = 0.2 meter
    
#plt.gca().add_artist(scalebar)

figure.add_subplot(133)
plt.imshow(ecm_image,interpolation='none',cmap='inferno',alpha=.5)
plt.imshow(endosome_image,interpolation='none',cmap='viridis',alpha=.5)
plt.title('endosome raw')
plt.grid(visible=False)
plt.xticks(visible=False)
plt.yticks(visible=False)
scalebar = ScaleBar(40, 'nm', dimension='si-length',box_alpha=0,location='lower right',length_fraction=0.2,
                       font_properties={'size':16,'weight':'bold'},color='white',border_pad=0.5) # 1 pixel = 0.2 meter
    
plt.gca().add_artist(scalebar)

NameError: name 'np' is not defined

In [8]:
blurred = ndi.gaussian_filter(ecm_image,sigma=3)
blurred /= blurred.max()
mean, std = numpy.mean(blurred), numpy.std(blurred)
blurred -= 1.2 * mean # heuristic that works
blurred /= std
blurred = numpy.maximum(blurred, 0)
val = filters.threshold_triangle(blurred)
a=ndi.binary_erosion(blurred > val, selem=morphology.square(selem))

TypeError: No loop matching the specified signature and casting
was found for ufunc true_divide

## Find spots

### Manual

In [241]:
#make empty masks
for i in range(0,len(results["filename"])):
    results["Manual ECM spots mask"].append(np.nan)
    results["Manual endosome spots mask"].append(np.nan)

In [266]:
import ipywidgets as widgets
from ipywidgets import interact
first=0

#load first image
ecm_channel_no=results["ecmchannelno"][first]
ecm_image_name=results["filename"][first]+"0_ch"+str(ecm_channel_no-1)+"_t0.tif"
ecm_image=pims.open(ecm_image_name)[0]
ecm_spots=deepcopy(ecm_image)
ecm_spots=ndi.median_filter(ecm_spots,size=5)

lower_man_threshold=np.min(ecm_spots)
upper_man_threshold=np.max(ecm_spots)

def f(Threshold,Manual_Threshold,Open,Close,Image_no):
    
    ii=Image_no
    ecm_channel_no=results["ecmchannelno"][ii]
    ecm_image_name=results["filename"][ii]+"0_ch"+str(ecm_channel_no-1)+"_t0.tif"
    ecm_image=pims.open(ecm_image_name)[0]
    #bandpass (subtract gaussian filter)
    #lowpass = ndi.gaussian_filter(ecm_image, 3)
    #ecm_image = ecm_image - lowpass
    #ecm_image[ecm_image<0]=0
    
    ecm_spots=deepcopy(ecm_image)
    lower_man_threshold=np.min(ecm_spots)
    upper_man_threshold=np.max(ecm_spots)

    if Threshold=="Li":
        ecm_image_threshold=threshold_li(ecm_image)      
    if Threshold=="Otsu":
        ecm_image_threshold=threshold_otsu(ecm_image)
    if Threshold=="Manual":
        ecm_image_threshold=Manual_Threshold
    
    ecm_spots[ecm_spots<ecm_image_threshold]=0
    ecm_spots=ndi.binary_opening(ecm_spots,iterations=Open)
    ecm_spots=ndi.binary_closing(ecm_spots,iterations=Close)
    results["Manual ECM spots mask"][ii]=ecm_spots
    fig=plt.figure(figsize=(10,10))
    fig.add_subplot(121)
    plt.imshow(ecm_spots)
    plt.imshow(ecm_image,alpha=0.5,cmap='inferno')
    fig.add_subplot(122)
    plt.imshow(ecm_image,cmap='inferno')
    

#widgets
Threshold=widgets.Dropdown(options=["Li","Otsu","Manual"])
Manual_Threshold=widgets.IntSlider(min=lower_man_threshold,max=upper_man_threshold,step=1,value=lower_man_threshold)
Open=widgets.IntText(min=0,max=20,step=1,value=1)
Close=widgets.IntText(min=0,max=20,step=1,value=1)
Image_no=widgets.IntText(min=0,max=len(results["filename"]),value=0)

w=interact(f,Threshold=Threshold,Manual_Threshold=Manual_Threshold,Open=Open,Close=Close,
           Image_no=Image_no)

ImageJ series: invalid metadata or corrupted file
TiffPages: invalid page offset (268088)


interactive(children=(Dropdown(description='Threshold', options=('Li', 'Otsu', 'Manual'), value='Li'), IntSlid…

### Backup manual spots for ecm

In [275]:
for image in range(0,len(results["filename"])):
    np.save(results["filename"][image]+'_ecm_spots_mask.npy',results["Manual ECM spots mask"][image])

### Make spots for endosome channel

In [272]:
import ipywidgets as widgets
from ipywidgets import interact
first=0

#load first image
endosome_channel_no=results["endosomechannelno"][first]
endosome_image_name=results["filename"][first]+"0_ch"+str(endosome_channel_no-1)+"_t0.tif"
endosome_image=pims.open(endosome_image_name)[0]
endosome_spots=deepcopy(endosome_image)
endosome_spots=ndi.median_filter(endosome_spots,size=5)

lower_man_threshold=np.min(endosome_spots)
upper_man_threshold=np.max(endosome_spots)

def f(Threshold,Manual_Threshold,Open,Close,Image_no):
    
    ii=Image_no
    endosome_channel_no=results["endosomechannelno"][ii]
    endosome_image_name=results["filename"][ii]+"0_ch"+str(endosome_channel_no-1)+"_t0.tif"
    endosome_image=pims.open(endosome_image_name)[0]
    #bandpass (subtract gaussian filter)
    #lowpass = ndi.gaussian_filter(endosome_image, 3)
    #endosome_image = endosome_image - lowpass
    #endosome_image[endosome_image<0]=0
    
    endosome_spots=deepcopy(endosome_image)
    lower_man_threshold=np.min(endosome_spots)
    upper_man_threshold=np.max(endosome_spots)

    if Threshold=="Li":
        endosome_image_threshold=threshold_li(endosome_image)      
    if Threshold=="Otsu":
        endosome_image_threshold=threshold_otsu(endosome_image)
    if Threshold=="Manual":
        endosome_image_threshold=Manual_Threshold
    
    endosome_spots[endosome_spots<endosome_image_threshold]=0
    endosome_spots=ndi.binary_opening(endosome_spots,iterations=Open)
    endosome_spots=ndi.binary_closing(endosome_spots,iterations=Close)
    results["Manual endosome spots mask"][ii]=endosome_spots
    fig=plt.figure(figsize=(10,10))
    fig.add_subplot(121)
    plt.imshow(endosome_spots)
    plt.imshow(endosome_image,alpha=0.5,cmap='inferno')
    fig.add_subplot(122)
    plt.imshow(endosome_image,cmap='inferno')
    

#widgets
Threshold=widgets.Dropdown(options=["Li","Otsu","Manual"])
Manual_Threshold=widgets.IntSlider(min=lower_man_threshold,max=upper_man_threshold,step=1,value=lower_man_threshold)
Open=widgets.IntText(min=0,max=20,step=1,value=1)
Close=widgets.IntText(min=0,max=20,step=1,value=1)
Image_no=widgets.IntText(min=0,max=len(results["filename"]),value=0)

w=interact(f,Threshold=Threshold,Manual_Threshold=Manual_Threshold,Open=Open,Close=Close,
           Image_no=Image_no)

ImageJ series: invalid metadata or corrupted file
TiffPages: invalid page offset (268088)


interactive(children=(Dropdown(description='Threshold', options=('Li', 'Otsu', 'Manual'), value='Li'), IntSlid…

In [274]:
for image in range(0,len(results["filename"])):
    np.save(results["filename"][image]+'_endosome_spots_mask.npy',results["Manual endosome spots mask"][image])

## Quick check coloc

In [273]:
#dictionary per image:
ecm_image_results={"filename":[],"exp condition":[],
                   "distances to nearest endosome boundary":[],
                  "distances to nearest endosome boundary [nms]":[],
                  "min distance to nearest endosome boundary for whole image":[],
                  "min distance to nearest endosome boundary for whole image [nms]":[],
                  "rotated - distances to nearest endosome boundary":[],
                  "rotated - distances to nearest endosome boundary [nms]":[],
                  "rotated - min distance to nearest endosome boundary for whole image":[],
                  "rotated - min distance to nearest endosome boundary for whole image [nms]":[]}

for image in range(0,len(results["filename"])):
    print('Calculating image no ' + str(image+1) + ' out of' + str(len(results["filename"])))
    
    #read spots masks and label
    ecm_mask=results["Manual ECM spots mask"][image]
    if len(ecm_mask.shape)==0:
        ecm_mask=np.zeros(ecm_image.shape).astype(bool)
    endosome_mask=results["Manual endosome spots mask"][image]
    if len(endosome_mask.shape)==0:
        endosome_mask=np.zeros(endosome_image.shape).astype(bool)
    #rotated ecm mask
    rot_ecm_mask=deepcopy(ecm_mask)
    rot_ecm_mask=np.flip(rot_ecm_mask)#np.rot90(rot_ecm_mask)
    labeled_ecm=label(ecm_mask)
    rot_labeled_ecm=label(rot_ecm_mask)
    labeled_endosome=label(endosome_mask)
    #regionprops for labeled masks
    if np.sum(ecm_mask>0) & np.sum(endosome_mask>0):
        regionprops_ecm=regionprops(labeled_ecm)
        regionprops_endosome=regionprops(labeled_endosome)
        rot_regionprops_ecm=regionprops(rot_labeled_ecm) 
        ecm_spots_results=[]
        rot_ecm_spots_results=[]
        endosome_spots_results=[]
        for i in range(0,len(regionprops_ecm)):
            ecm_spots_results.append(regionprops_ecm[i]['centroid'])
        for i in range(0,len(rot_regionprops_ecm)):
            rot_ecm_spots_results.append(rot_regionprops_ecm[i]['centroid'])
        for i in range(0,len(regionprops_endosome)):
            endosome_spots_results.append(regionprops_endosome[i]['coords'])
        #minimum distance between spots
        distances=[]
        rot_distances=[]
        for blob in range(0,len(endosome_spots_results)):
            distances.append(scipy.spatial.distance.cdist
                             (np.round(ecm_spots_results),endosome_spots_results[blob],'euclidean'))
            rot_distances.append(scipy.spatial.distance.cdist
                             (np.round(rot_ecm_spots_results),endosome_spots_results[blob],'euclidean'))
        #make array of min distance from each ecm centroid
        ecm_min_distances_array=np.amin(distances[0],1,keepdims=True)
        for blob in range(1,len(endosome_spots_results)):
            newmins=np.amin(distances[blob],1,keepdims=True)
            concat=np.concatenate([deepcopy(ecm_min_distances_array),newmins],1)
            ecm_min_distances_array=np.amin(deepcopy(concat),1,keepdims=True)
        rot_ecm_min_distances_array=np.amin(rot_distances[0],1,keepdims=True)
        for blob in range(1,len(endosome_spots_results)):
            newmins=np.amin(rot_distances[blob],1,keepdims=True)
            concat=np.concatenate([deepcopy(rot_ecm_min_distances_array),newmins],1)
            rot_ecm_min_distances_array=np.amin(deepcopy(concat),1,keepdims=True)

        ecm_image_results["filename"].append(results["filename"][image])
        ecm_image_results["exp condition"].append(results["exp condition"][image])
        ecm_image_results["distances to nearest endosome boundary"].append(ecm_min_distances_array)
        ecm_image_results["rotated - distances to nearest endosome boundary"].append(rot_ecm_min_distances_array)
        ecm_image_results["distances to nearest endosome boundary [nms]"].append(ecm_min_distances_array*40.4)
        ecm_image_results["rotated - distances to nearest endosome boundary [nms]"].append(rot_ecm_min_distances_array*40.4)
        ecm_image_results["min distance to nearest endosome boundary for whole image"].append(np.min(ecm_min_distances_array))
        ecm_image_results["min distance to nearest endosome boundary for whole image [nms]"].append(np.min(ecm_min_distances_array)*40.4)
        ecm_image_results["rotated - min distance to nearest endosome boundary for whole image"].append(np.min(rot_ecm_min_distances_array))
        ecm_image_results["rotated - min distance to nearest endosome boundary for whole image [nms]"].append(np.min(rot_ecm_min_distances_array)*40.4)
        
df_ecm=pd.DataFrame(ecm_image_results)
df_ecm['Protein']='None'
df_ecm.loc[(df_ecm['exp condition'].str.contains('nt'))&(df_ecm['exp condition'].str.contains('12')), 'Protein'] = 'Int1b, all TNR Blk+12, stripped'
df_ecm.loc[(df_ecm['exp condition'].str.contains('nt'))&(df_ecm['exp condition'].str.contains('surface')), 'Protein'] = 'Int1b Blk/TNR Blk, +0, surface'
df_ecm.loc[df_ecm['exp condition'].str.contains('tgn'), 'Protein'] = 'TGN38/ TNR Blk, +12'
df_ecm.loc[df_ecm['exp condition'].str.contains('rab11'), 'Protein'] = 'Rab11a/ TNR Blk, +12'
df_ecm.loc[df_ecm['exp condition'].str.contains('cav'), 'Protein'] = 'Cav1/ TNR Blk'
df_ecm.loc[df_ecm['exp condition'].str.contains('negctrl'), 'Protein'] = 'Neg ctrl vAchT'
df_ecm.loc[df_ecm['exp condition'].str.contains('positivecontrol'), 'Protein'] = 'Pos ctrl 2 secondaries'
df_ecm.loc[df_ecm['exp condition'].str.contains('rab7'), 'Protein'] = 'Rab7/ TNR Blk, +12'
df_ecm.loc[df_ecm['exp condition'].str.contains('rab5'), 'Protein'] = 'Rab5/ TNR Blk, +12'
df_ecm.loc[df_ecm['exp condition'].str.contains('lamp'), 'Protein'] = 'lamp1/ TNR Blk, +12'

spots_list={"filename":[],"expname":[],"Protein":[],"distance to nearest endosome boundary [nms]":[],
            "rotated - distance to nearest endosome boundary [nms]":[]}

for i in range(0,len(df_ecm)):
    nospots=len(df_ecm["distances to nearest endosome boundary [nms]"][i])
    protein=df_ecm["Protein"][i]
    file=df_ecm["filename"][i]
    exp=df_ecm["exp condition"][i]
    for spot in range(0,nospots):
        spots_list["filename"].append(file)
        spots_list["expname"].append(exp)
        spots_list["Protein"].append(protein)
        spots_list["distance to nearest endosome boundary [nms]"].append(
            df_ecm["distances to nearest endosome boundary [nms]"][i][spot][0])
        spots_list["rotated - distance to nearest endosome boundary [nms]"].append(
            df_ecm["rotated - distances to nearest endosome boundary [nms]"][i][spot][0])
        
dfspots=pd.DataFrame(spots_list)

#spots dictionary
exps_under={"proteinl":[],"percentunderl":[],"expname":[]}
              
nanometers=200

for exp in dfspots["expname"].unique():
    dfexp=dfspots[dfspots["expname"]==exp].reset_index()
    for protein in dfexp["Protein"].unique():
        df2plot=dfexp[dfexp["Protein"]==protein]
        exps_under["expname"].append(exp)
        exps_under["percentunderl"].append(np.sum(df2plot["distance to nearest endosome boundary [nms]"]<nanometers)/len(df2plot["distance to nearest endosome boundary [nms]"])*100)
        exps_under["proteinl"].append(protein)
        #exps_under["percentunderl"].append(percent_under*100)

dfexps_under=pd.DataFrame(exps_under)
dfexps_under


Calculating image no 1 out of9
Calculating image no 2 out of9
Calculating image no 3 out of9
Calculating image no 4 out of9
Calculating image no 5 out of9
Calculating image no 6 out of9
Calculating image no 7 out of9
Calculating image no 8 out of9
Calculating image no 9 out of9


,proteinl,percentunderl,expname
0,"Int1b Blk/TNR Blk, +0, surface",14.840989,20190515_tnr550blocked_int1fitcblocked_plus0h_...


In [201]:
results["filename"]

['/Volumes/ukps-all$/user/tdanko/Imaging/1_ECM/Fixed culture/Leica confocal/20190612_tnr550_blkplus12_tgn38acy2_analyse/20190612_tnr550_blkplus12_tgn38acy2_analyse/20190612_tnr550_blkplus12_tgn38acy2_Series8_z',
 '/Volumes/ukps-all$/user/tdanko/Imaging/1_ECM/Fixed culture/Leica confocal/20190612_tnr550_blkplus12_tgn38acy2_analyse/20190612_tnr550_blkplus12_tgn38acy2_analyse/20190612_tnr550_blkplus12_tgn38acy2_Series9_z',
 '/Volumes/ukps-all$/user/tdanko/Imaging/1_ECM/Fixed culture/Leica confocal/20190612_tnr550_blkplus12_tgn38acy2_analyse/20190612_tnr550_blkplus12_tgn38acy2_analyse/20190612_tnr550_blkplus12_tgn38acy2_Series0_z',
 '/Volumes/ukps-all$/user/tdanko/Imaging/1_ECM/Fixed culture/Leica confocal/20190612_tnr550_blkplus12_tgn38acy2_analyse/20190612_tnr550_blkplus12_tgn38acy2_analyse/20190612_tnr550_blkplus12_tgn38acy2_Series1_z',
 '/Volumes/ukps-all$/user/tdanko/Imaging/1_ECM/Fixed culture/Leica confocal/20190612_tnr550_blkplus12_tgn38acy2_analyse/20190612_tnr550_blkplus12_tgn38a